In [ ]:
!pip install selenium
!pip install kora -q

In [ ]:
import numpy as np
import pandas as pd
import re
import requests
import time

In [3]:
from bs4 import BeautifulSoup
from kora.selenium import wd
from selenium.common.exceptions import NoSuchElementException
import pickle


In [4]:
years = range(1964, 2021)
song_id_regex = re.compile('[0-9]+')
base_url = "https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate="
songList = pd.DataFrame(columns=['year', 'title', 'artist', 'song_id'])


In [ ]:
for year in years:
  wd.get(base_url+str(year))
  chart = wd.find_element_by_css_selector("form#frm")
  current = pd.DataFrame(
      { 
          "year" : year, 
          "title": [ 
                    title.text.strip() 
                    for title in (
                        chart.find_element_by_css_selector("form#frm table")
                        .find_elements_by_css_selector("div.wrap_song_info div.rank01")
                    )
          ],
          "artist": [
                    artist.get_attribute("textContent")
                    for artist in (
                        chart.find_element_by_css_selector("form#frm table")
                        .find_elements_by_css_selector("span.checkEllipsis")
                    )
          ],
          "song_id": [
                      song_id_regex.findall(song_id.get_attribute('onclick'))[0]
                      for song_id in (
                          chart.find_element_by_css_selector("form#frm table")
                          .find_elements_by_css_selector("div.wrap a.btn")
                      )
          ],
      }
  )
  songList = pd.concat((songList, current))
  time.sleep(60)
  

In [6]:
details = pd.DataFrame(columns=['lyric', 'genre'])
detail_base_url = "http://www.melon.com/song/detail.htm?songId="
lyric_tag_regex = re.compile("<.*?>")
song_ids = songList['song_id']


In [ ]:
for song_id in song_ids:
  wd.get(detail_base_url + song_id)

  try:
    lyric = wd.find_element_by_css_selector('div.lyric').text
  except NoSuchElementException:
    pass
  try:
    genre = (
        wd.find_element_by_css_selector('div.meta')
        .find_elements_by_tag_name('dd')[2]
        .text
    )
  except NoSuchElementException:
    pass

  current = pd.DataFrame(
      {
          "song_id": [song_id],
          "lyric": [lyric],
          "genre": [genre],
      }
  )
  details = pd.concat((details, current))
  time.sleep(1)


In [8]:
song_list = pd.merge(songList, details, left_on='song_id', right_on='song_id', how='inner')

In [11]:
song_list.to_csv('melon_lyrics_1964-2020.csv')

In [ ]:
from google.colab import files
files.download('melon_lyrics_1964-2020.csv')

In [ ]:
!pwd